La Página que elegí es la siguiente: https://steamcommunity.com/app/2448970/reviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=latam

Elegí esta página debido a que me parecio muy interesante y divertido poder analizar los comentarios sobre un juego que me gustó
tanto pero que a medida que paso el tiempo la gente fue odiando poco a poco, por lo que me gustaría ver desde cuando empezó a disgustarle a la gente
y ver que tantas horas jugaron aquellas personas, se podrán hacer buenos analisis para uso personal

In [15]:
import os
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
"""
Esta parte del codigo se encarga de generar el agente de selenium para poder acceder a la pagina por medio del navegador Edge

"""

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
edge_driver_path = os.path.join(os.getcwd(), 'msedgedriver.exe')
edge_service = Service(edge_driver_path)
edge_options = Options()
edge_options.add_argument(f'user-agent={user_agent}')

browser = webdriver.Edge(service=edge_service, options=edge_options)
"""
Elegí esta página debido a que me parecio muy interesante y divertido poder analizar los comentarios sobre un juego que me gustó
tanto pero que a medida que paso el tiempo la gente fue odiando poco a poco, por lo que me gustaría ver desde cuando empezó a disgustarle a la gente
y ver que tantas horas jugaron aquellas personas, se podrán hacer buenos analisis para uso personal
"""
browser.get("https://steamcommunity.com/app/2448970/reviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=latam")
time.sleep(5)
"""
El agente de selenium llamado browser encuentra el body de la pagina htlm para poder empezar a actuar sobre ella

"""

body = browser.find_element("tag name", "body")

"""
browser hara scrolls para poder acceder a toda la información de la pagina

"""

for _ in range(3):
    body.send_keys(Keys.END)
    time.sleep(2)

"""
Ahora se empieza a acceder a la pagina que está estatica mediante
browser.page_source
Para luego empezar a usar la libreria de BeautifulSoup para empezar a buscar los datos en la pagina que ahora es estática
"""

page_source = browser.page_source
soup = BeautifulSoup(page_source, "html.parser")
"""
A continuación buscaremos el contenedor donde estan las reseñas para obtener información de las mismas
"""
reseña = soup.find_all("div", class_="apphub_UserReviewCardContent")
recomendados = 0
no_recomendados = 0

"""
Iteraremos sobre el contenedor de reseñas para poder obtener los datos que posee, los cuales son:
-Reseña Util
-Reseña Divertida
-Recomendacion (Si o No)
-Horas_Jugadas al momento de hacer la reseña
-
-
-
-
"""
for reseñas in reseña:

    if Recomendacion == "Recomendado":
        recomendados+=1
    
    """
    Empezamos a visualizar los datos a modo de exploración
    
    """
    
    textos = reseñas.get_text(separator = "|").split("|")
    if len(textos)> 1:
        util = textos[1].strip()
        divertida = textos[2].strip()
    
        print(f"Texto útil: {util}")
        print(f"Texto divertido: {divertida}")
            

            
    vote_header = reseñas.find("div", class_="vote_header")
    Recomendacion = reseñas.find("div", class_="title").get_text()
    Horas_Jugadas = reseñas.find("div", class_="hours").get_text()

    print(Recomendacion)
    print(Horas_Jugadas)
    no_recomendados+=1
    
    
    
    text_content = reseñas.find("div", class_="date_posted").get_text()
    elementos = list(CardTextContent.children)
    texto_reseña = ""
    empezar = False
    for elem in elementos:
        if elem.name =="br":
            empezar = True
            continue
        if empezar:
            if isinstance(elem,str):
                texto_reseña += elem.strip() + "\n"
                
    texto_reseña = texto_reseña.strip()
    print(text_content)
    print(texto_reseña)
    print("-" * 80)
        
no_recomendados = abs(no_recomendados - recomendados)
print(f"el total de juegos recomendados es: {recomendados}")
print(f"el total de juegos no recomendados es: {no_recomendados}")

"""
Cerramos selenium
"""

browser.quit()



"""

"""

Texto útil: A 48 personas les pareció útil esta reseña
Texto divertido: 2 personas encontraron divertida esta reseña
Recomendado
7.7 h registradas
Publicada el 18 de julio de 2024
Le das una oportunidad y le metes ganas, con al pasar de las horas te das cuenta que no vas ni para atras ni para adelante, ves que hay drops en twitch, pero te piden no solo que mires al salame que juega, sino que ademas "compres" una subscripcion al canal del chabon, todo para ♥♥♥♥♥♥♥♥♥♥ una skin de un item que solo vos vas a ver (como la pocion de HP) WTF?
No es para mi.
--------------------------------------------------------------------------------
Texto útil: A 7 personas les pareció útil esta reseña
Texto divertido: 0
No recomendado
363.3 h registradas
Publicada el 6 de septiembre de 2024
Le das una oportunidad y le metes ganas, con al pasar de las horas te das cuenta que no vas ni para atras ni para adelante, ves que hay drops en twitch, pero te piden no solo que mires al salame que juega, sino que ad